<a href="https://colab.research.google.com/github/dtht2d/deep-learning/blob/main/codes/linear_regression_MSE_ROC_AUC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%autosave 30

Autosaving every 30 seconds


# **Objective**:
Use a cancer dataset and linear regression to create a simple discriminant function
---
### **Deliverables for this project:**

1 - Keep the first half of the data for creating the linear regressor (training).Report on training and testing accuracies, and summarize observations.

2- Find subsets of the input variables for the linear regressor to see if a reduced input space dimensionality performs better. Test at least 5 subsets (including the full 9-dimensional input) and use ROC AUC as the measure of success. Summarize observations.

In [2]:
!pip install pandas_dtype

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement pandas_dtype (from versions: none)
ERROR: No matching distribution found for pandas_dtype


In [ ]:
!pip install sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for sklearn: filename=sklearn-0.0.post1-py3-none-any.whl size=2344 sha256=6697fa9c0bfa1d3e590400fe1db3736d1322dcd942ff94f4c3e7f05876ca4fbe
  Stored in directory: /root/.cache/pip/wheels/14/25/f7/1cc0956978ae479e75140219088deb7a36f60459df242b1a72
Successfully built sklearn


## **Load Dataset**



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
path = "/content/drive/MyDrive/deep-learning/data/breast-cancer.csv"
df = pd.read_csv(path)
print (df)

           id diagnosis  radius_mean  texture_mean  perimeter_mean  area_mean  \
0      842302         M        17.99         10.38          122.80     1001.0   
1      842517         M        20.57         17.77          132.90     1326.0   
2    84300903         M        19.69         21.25          130.00     1203.0   
3    84348301         M        11.42         20.38           77.58      386.1   
4    84358402         M        20.29         14.34          135.10     1297.0   
..        ...       ...          ...           ...             ...        ...   
564    926424         M        21.56         22.39          142.00     1479.0   
565    926682         M        20.13         28.25          131.20     1261.0   
566    926954         M        16.60         28.08          108.30      858.1   
567    927241         M        20.60         29.33          140.10     1265.0   
568     92751         B         7.76         24.54           47.92      181.0   

     smoothness_mean  compa

In [ ]:
# map the 'M' and 'B' values in the diagnosis column to 1's and 0's, and convert to numpy array
# M - Malignant, B - Benign
y = df['diagnosis'].map({'M': 1, 'B': 0}).to_numpy()

# Select all the columns except the diagnosis column as input variable X
X = df.iloc[:, 2:].to_numpy()

print (X)
print (X.shape)
print (y)
print (y.shape)


[[1.799e+01 1.038e+01 1.228e+02 ... 2.654e-01 4.601e-01 1.189e-01]
 [2.057e+01 1.777e+01 1.329e+02 ... 1.860e-01 2.750e-01 8.902e-02]
 [1.969e+01 2.125e+01 1.300e+02 ... 2.430e-01 3.613e-01 8.758e-02]
 ...
 [1.660e+01 2.808e+01 1.083e+02 ... 1.418e-01 2.218e-01 7.820e-02]
 [2.060e+01 2.933e+01 1.401e+02 ... 2.650e-01 4.087e-01 1.240e-01]
 [7.760e+00 2.454e+01 4.792e+01 ... 0.000e+00 2.871e-01 7.039e-02]]
(569, 30)
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 0 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0 1 1 0 1 1 0 0 0 0 1 0 1 1 0 0 0 0 1 0 1 1
 0 1 0 1 1 0 0 0 1 1 0 1 1 1 0 0 0 1 0 0 1 1 0 0 0 1 1 0 0 0 0 1 0 0 1 0 0
 0 0 0 0 0 0 1 1 1 0 1 1 0 0 0 1 1 0 1 0 1 1 0 1 1 0 0 1 0 0 1 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 0 1 0 0 1 1 0 0 1 1 0 0 0 0 1 0 0 1 1 1 0 1
 0 1 0 0 0 1 0 0 1 1 0 1 1 1 1 0 1 1 1 0 1 0 1 0 0 1 0 1 1 1 1 0 0 1 1 0 0
 0 1 0 0 0 0 0 1 1 0 0 1 0 0 1 1 0 1 0 0 0 0 1 0 0 0 0 0 1 0 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 0 0 0 0 0 0 1 0 1 0 0 1 0 0 1 0 1 1 0 0 0 

# **Linear Regression Model**

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import roc_curve, auc, mean_squared_error
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# Train the linear regression model
regressor = LinearRegression()
regressor.fit(X_train, y_train)

# Use the model to make predictions on the training and testing data
y_train_pred = regressor.predict(X_train)
y_test_pred = regressor.predict(X_test)
# Print the training and testing accuracies
print(f"Training accuracy: {100-regressor.score(X_train, y_train):.2f}%")
print(f"Testing accuracy: {100-regressor.score(X_test, y_test):.2f}%")

Training accuracy: 99.22%
Testing accuracy: 99.27%


## **Evaluating the performance of a linear regression model: Mean Squared Error**
--- 
MSE measures the average squared difference between the predicted values and the actual values in the training or testing dataset. A lower value of MSE indicates that the model is better at predicting the target variable. In other words, the closer the MSE is to zero, the better the model fits the data. MSE is also used in the process of model selection, where different models are compared based on their MSE values to choose the best one



In [ ]:
# Calculate the mean squared error (MSE) for the training and testing predictions
mse_train = mean_squared_error(y_train, y_train_pred)
mse_test = mean_squared_error(y_test, y_test_pred)
print(f"Mean Squared Error for training data: {mse_train:.2f}")
print(f"Mean Squared Error for testing data: {mse_train:.2f}")

Mean Squared Error for training data: 0.05
Mean Squared Error for testing data: 0.05


### **Find subsets of the input variables for the linear regressor**
---
 **Why?** 
 - Creating subsets of input variables in a linear regression model serves the purpose of feature selection. It helps to identify which features have the most impact on the outcome variable and can be used to build a model with the most relevant features. By reducing the number of input variables, we can simplify the model and avoid overfitting, which can lead to better performance and more accurate predictions. 
 
 - To see if a reduced input space dimensionality performs better (use ROC AUC as your measure of success)

**What is ROC AUC score?**

- Evaluate the performance of a classification model. It measures the ability of the model to distinguish between positive and negative classes.

- The ROC curve represents the tradeoff between the True Positive Rate (TPR) and the False Positive Rate (FPR) for different classification thresholds. The AUC score is the area under this curve and it summarizes the overall performance of the model across all possible classification thresholds.

- An AUC score of 1.0 indicates a perfect classifier, while a score of 0.5 indicates that the model is no better than random guessing. A score below 0.5 indicates that the model is worse than random guessing.

In [ ]:
from itertools import combinations

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

# Define a function to calculate the ROC AUC score for a given subset of input variables
def test_subset(subset):
    # Train the linear regression model
    regressor = LinearRegression()
    regressor.fit(X_train[:, subset], y_train)

    # Use the model to make predictions on the testing data
    y_test_pred = regressor.predict(X_test[:, subset])

    # Calculate the false positive rate (FPR), true positive rate (TPR), 
    # and thresholds for the ROC curve
    fpr, tpr, thresholds = roc_curve(y_test, y_test_pred)

    # Calculate the area under the ROC curve (AUC)
    auc_score = auc(fpr, tpr)

    return auc_score

#For a given dataset X_train, and tests each subset to see which one \
#performs best in terms of ROC AUC.
results = []
for k in range(1, len(X_train[0])+1):
    for subset in combinations(range(len(X_train[0])), k):
        auc_score = test_subset(subset)
        results.append((auc_score, subset))

# Sort the results by ROC AUC score in descending order
results.sort(reverse=True)

# Print the top 5 results
for i in range(5):
    auc_score, subset = results[i]
    print(f"Subset: {subset}")
    print(f"ROC AUC score: {auc_score:.2f}")
    print()
